Project: /meridian/_project.yaml
Book: /meridian/_book.yaml

<style>
devsite-code .tfo-notebook-code-cell-output {
  max-height: 300px;
  overflow: auto;
  background: rgba(255, 247, 237, 1);  /* orange bg to distinguish from input code cells */
}

devsite-code .tfo-notebook-code-cell-output + .devsite-code-buttons-container button {
  background: rgba(255, 247, 237, .7);  /* orange bg to distinguish from input code cells */
}

devsite-code[dark-code] .tfo-notebook-code-cell-output {
  background: rgba(64, 78, 103, 1);  /* medium slate */
}

devsite-code[dark-code] .tfo-notebook-code-cell-output + .devsite-code-buttons-container button {
  background: rgba(64, 78, 103, .7);  /* medium slate */
}

/* override default table styles for notebook buttons */
.devsite-table-wrapper .tfo-notebook-buttons {
  display: inline-block;
  margin-left: 3px;
  width: auto;
}

.tfo-notebook-buttons tr {
  background: 0;
  border: 0;
}

.tfo-notebook-buttons td {
  padding-left: 0;
  padding-right: 20px;
}

.tfo-notebook-buttons {
  --tfo-notebook-buttons-box-shadow: 0 1px 2px 0 rgba(60, 64, 67, .3), 0 1px 3px 1px rgba(60, 64, 67, .15);
}

.tfo-notebook-buttons a,
.tfo-notebook-buttons :link,
.tfo-notebook-buttons :visited {
  border-radius: 8px;
  box-shadow: var(--tfo-notebook-buttons-box-shadow);
  color: #202124;
  padding: 12px 24px;
  transition: box-shadow 0.2s;
}

.tfo-notebook-buttons a:hover,
.tfo-notebook-buttons a:focus {
  box-shadow: var(--tfo-notebook-buttons-box-shadow);
}

.tfo-notebook-buttons td > a {
  -webkit-box-align: center;
  -ms-flex-align: center;
  align-items: center;
  display: -webkit-box;
  display: -ms-flexbox;
  display: flex;
}

.tfo-notebook-buttons td > a > img {
  margin-right: 8px;
}
</style>

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/meridian/blob/main/demo/ROI_mROI_Response_Curves.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/meridian/blob/main/demo/ROI_mROI_Response_Curves.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# **ROI, mROI & Response Curves in Meridian**

This notebook showcases how to extract and interpret core business metrics from a trained Meridian model, including **Return on Investment (ROI)**, **marginal ROI (mROI)**, and **Response Curves**.

For a detailed overview on each metric\, please visit this [page](https://developers.google.com/meridian/docs/basics/roi-mroi-response-curves).

This notebook utilizes sample data. As a result, the numbers and results obtained might not accurately reflect what you encounter when working with a real dataset.


<ol start="0">
  <li><a href="#install">Preliminary Setup</a></li>
  <li><a href="#point-estimates">Point Estimates</a></li>
  <li><a href="#credible-intervals">Credible Intervals</a></li>
  <li><a href="#roi-mroi-distributions">ROI, mROI distributions</a></li>
  <li><a href="#response-curves">Response Curves</a></li>
</ol>



<a name="install"></a>
## Preliminary setup: Install library, import data, train model

This part of the notebook is preliminary setup to install the Meridian Library and train the model. The core part of the notebook involves understanding the output from the model.

1. Make sure you are using one of the available GPU Colab runtimes which is **required** to run Meridian. You can change your notebook's runtime in `Runtime > Change runtime type` in the menu. All users can use the T4 GPU runtime which is sufficient to run the demo colab, free of charge. Users who have purchased one of Colab's paid plans have access to premium GPUs (such as V100, A100 or L4 Nvidia GPU).

2. Install the latest version of Meridian, and verify that GPU is available.

### Install library

In [ ]:
# Install meridian: from PyPI @ latest release
!pip install --upgrade google-meridian[colab,and-cuda]

import arviz as az
import IPython
from meridian import constants
from meridian.analysis import analyzer
from meridian.analysis import optimizer
from meridian.analysis import summarizer
from meridian.analysis import visualizer
from meridian.data import data_frame_input_data_builder
from meridian.data import test_utils
from meridian.model import model
from meridian.model import prior_distribution
from meridian.model import spec
from meridian.templates import formatter
import numpy as np
import pandas as pd
# check if GPU is available
from psutil import virtual_memory
import tensorflow as tf
import tensorflow_probability as tfp

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print(
    'Num GPUs Available: ',
    len(tf.config.experimental.list_physical_devices('GPU')),
)
print(
    'Num CPUs Available: ',
    len(tf.config.experimental.list_physical_devices('CPU')),
)

### Load Data

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/geo_all_channels.csv"
)

builder = data_frame_input_data_builder.DataFrameInputDataBuilder(
    kpi_type='non_revenue',
    default_kpi_column='conversions',
    default_revenue_per_kpi_column='revenue_per_conversion',
)

builder = (
    builder.with_kpi(df)
    .with_revenue_per_kpi(df)
    .with_population(df)
    .with_controls(
        df, control_cols=["sentiment_score_control", "competitor_sales_control"]
    )
)

channels = ["Channel0", "Channel1", "Channel2", "Channel3", "Channel4"]
builder = builder.with_media(
    df,
    media_cols=[f"{channel}_impression" for channel in channels],
    media_spend_cols=[f"{channel}_spend" for channel in channels],
    media_channels=channels,
)

data = builder.build()

<a name="configure-model"></a>
### Run the Model

We will initialize the `Meridian` class and use the `sample_posterior()` method to obtain samples from the posterior distribution.

In [ ]:
# Initialize and run the model
mmm = model.Meridian(input_data=data)
mmm.sample_posterior(
    n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=0
)


<a name="roi-mroi-response-curve"></a>

## Step 1: Analyze ROI, mROI & Respose Curves

Now that the model has been trained, we can use the `Analyzer` class to extract key business metrics. This class is the primary tool for translating the statistical model output into actionable marketing insights.

In [ ]:
# Create an instance of the Analyzer class
analyzer_instance = analyzer.Analyzer(mmm)

<a name="point-estimates"></a>
### Getting Point Estimates for Quick Analysis

We can calculate the **mean** of the posterior distribution to get a point estimate for each channel's ROI and mROI.

In [ ]:
analyzer_instance.summary_metrics().roi.sel({"distribution": "posterior", "metric": "mean"}).to_dataframe()

,metric,distribution,roi
channel,,,
Channel0,mean,posterior,1.586766
Channel1,mean,posterior,0.936712
Channel2,mean,posterior,1.418973
Channel3,mean,posterior,0.876884
Channel4,mean,posterior,1.620968
All Channels,mean,posterior,1.208054


In [ ]:
analyzer_instance.summary_metrics().mroi.sel({"distribution": "posterior", "metric": "mean"}).to_dataframe()

,metric,distribution,mroi
channel,,,
Channel0,mean,posterior,0.789475
Channel1,mean,posterior,0.470342
Channel2,mean,posterior,0.791703
Channel3,mean,posterior,0.411681
Channel4,mean,posterior,0.741940
All Channels,mean,posterior,NaN


### Understanding Uncertainty: Credible Intervals

A key benefit of Bayesian modeling is measuring uncertainty with credible intervals. A credible interval has a direct, intuitive interpretation. Unlike a frequentist confidence interval, Bayesian models make probabilistic statements about the parameters themselves, rather than the data. For example, a 95% credible interval of [\$0.43, \$3.15] for TV ROI means there is a 95% probability that the true ROI for TV is between \$0.43 and \$3.15. Such direct probability statements are useful for communicating and decision-making.

In [ ]:
analyzer_instance.summary_metrics().roi.sel({"distribution": "posterior"}).drop_vars("distribution").to_dataframe().unstack()

roi                              
metric            mean    median     ci_lo     ci_hi
channel                                             
Channel0      1.586766  1.463530  0.431896  3.155697
Channel1      0.936712  0.815354  0.261797  1.977477
Channel2      1.418973  1.346149  0.543670  2.552366
Channel3      0.876884  0.802209  0.297994  1.710027
Channel4      1.620968  1.495501  0.405906  3.291715
All Channels  1.208054  1.200087  0.800349  1.648818

<a name="roi-mroi-distributions"></a>

### ROI and mROI Posterior Distributions as Tensors

A key benefit of Meridian's Bayesian approach is that it doesn't just give you one number for ROI. Instead, it gives you a full **posterior distribution** for each metric, which represents the range of likely values given your data.

We can get these distributions for both ROI and mROI using the `.roi()` and `.marginal_roi()` methods.

In [ ]:
# Calculate ROI and mROI posterior distributions (as TensorFlow Tensors)
mroi_dist = analyzer_instance.marginal_roi()
roi_dist = analyzer_instance.roi()

print("Shape of the ROI posterior distribution tensor:", roi_dist.shape)
print("Shape of the mROI posterior distribution tensor:", mroi_dist.shape)
print("\nThe shape represents (Number of Chains, Number of Draws, Number of Geos, Number of Channels)")

Shape of the ROI posterior distribution tensor: (10, 500, 5)
Shape of the mROI posterior distribution tensor: (10, 500, 5)

The shape represents (Number of Chains, Number of Draws, Number of Geos, Number of Channels)


<a name="credible-intervals"></a>
### Standard Deviations of mROI & ROI Posterior Distributions

The full posterior distribution can be used to construct any summary metrics of interest. For example, the posterior standard deviation can be obtained as follows.

In [ ]:
roi_stdev = np.std(roi_dist, (0, 1))
mroi_stdev = np.std(mroi_dist, (0, 1))

stdev_df = pd.DataFrame({
    "ROI Stdev": roi_stdev,
    "MROI Stdev": mroi_stdev
    }, index=channels)
print("Posterior Standard Deviations:")
print(stdev_df)

Posterior Standard Deviations:
          ROI Stdev  MROI Stdev
Channel0   0.860600    0.442567
Channel1   0.557213    0.280965
Channel2   0.620319    0.367566
Channel3   0.438754    0.208942
Channel4   0.903968    0.394786


### Posterior Probability of ROI > 1

Another useful metric is the posterior probability that a channel's ROI is greater than 1. This directly tells us the probability that a channel is profitable, given the data and model.



In [ ]:
(mmm.inference_data.posterior.roi_m >= 1).mean(dim=('chain', 'draw'))

<xarray.DataArray 'roi_m' (media_channel: 5)> Size: 40B
array([0.72  , 0.3732, 0.735 , 0.3344, 0.7298])
Coordinates:
  * media_channel  (media_channel) object 40B 'Channel0' ... 'Channel4'

<a name="response-curves"></a>
### Visualizing Performance with Response Curves

**Response curves** help you visualize how each channel saturates as a function of spend. The curve shows how the incremental outcome (revenue, in this example) for a channel increases as you spend more. A flattening curve indicates diminishing returns.

We can plot these curves easily using the `visualizer` module.

In [ ]:
# Create a MediaEffects visualizer object
media_effects = visualizer.MediaEffects(mmm)

# Plot the response curves for each channel
media_effects.plot_response_curves(
    plot_separately=True,
    include_ci=True # Shows the credible interval on the curve
)

alt.FacetChart(...)